# How to Stacking Machine Learning Models

Tutorial for improve skills: 'Tutorial: Increasing the Predictive Power of Your Machine Learning Models with Stacking Ensembles' (Mario Filho - Data Science) by Marcus Mariano

**For more information about Marcus Mariano: [Web site](https://marcusmariano.github.io/mmariano/)**  

**Tutorial: Increasing the Predictive Power of Your Machine Learning Models with Stacking Ensembles [Mario Filho - Data Science.](https://github.com/ledmaster/TutorialEnsemble/blob/master/HomePrices-English.ipynb)**  

**House Prices Advanced Regression Techniques: [Kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/)**


## Tutorial: Increasing the Predictive Power of Your Machine Learning Models with Stacking Ensembles

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt

from tqdm.notebook import tqdm, tqdm_notebook

sns.set(style="darkgrid", color_codes=True)
%matplotlib inline

## Loading data

The data used are commercial real estate transactions in the city of Ames, Iowa. This data is also the subject of a competition at Kaggle. The training and test data can be found at: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/


In [3]:
train = pd.read_csv('data/train.csv', index_col='Id')

In [4]:
train = pd.read_csv('data/train.csv', index_col='Id')
X, y = train.drop('SalePrice', axis=1), train.SalePrice.copy()

train.head().iloc[:, :5]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street
Id,,,,,
1,60,RL,65.0,8450,Pave
2,20,RL,80.0,9600,Pave
3,60,RL,68.0,11250,Pave
4,70,RL,60.0,9550,Pave
5,60,RL,84.0,14260,Pave


In [3]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [6]:
y[10:]

Id
11      129500
12      345000
13      144000
14      279500
15      157000
         ...  
1456    175000
1457    210000
1458    266500
1459    142125
1460    147500
Name: SalePrice, Length: 1450, dtype: int64

## Sets of Variables (Features)

One way to get different models is to vary the data representation used to train them. That is why our first step is to build these representations.

In [5]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn.ensemble import RandomForestRegressor

In [6]:
def rmsle(estimator, X, y):
    p = estimator.predict(X)
    return np.sqrt(mean_squared_error(np.log1p(y), np.log1p(p)))

def rmsle_log_y(estimator, X, y):
    p = estimator.predict(X)
    return np.sqrt(mean_squared_error(y, p))

def rmsle_sqrt_y(estimator, X, y):
    p = estimator.predict(X)
    y = np.power(y, 2)
    p = np.power(p, 2)
    return np.sqrt(mean_squared_error(np.log1p(y), np.log1p(p)))

kf = KFold(n_splits=5, shuffle=True, random_state=1)

### Feature set 1: "numeric" variables


In [7]:
from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...


X1 = X.select_dtypes(include=[np.number]).fillna(-1)

print('Dims', X1.shape)

model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, X1, y, cv=kf, scoring=rmsle).mean()

print('RMSLE:', error)


# End Measure time elapsed
end = timer()
print('\n')
print('Time Elapsed: {}'.format(timedelta(seconds = end - start)))

Dims (1460, 36)
RMSLE: 0.14582352617986846


Time Elapsed: 0:01:19.794873


### Feature set 2: Ordinal Encoding Categorical

In [9]:
from sklearn.preprocessing import LabelEncoder

In [10]:
from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...


X2 = X.copy()
for col in X2.columns:
    if X2[col].dtype == object:
        enc = LabelEncoder()
        X2[col] = enc.fit_transform(X[col].fillna('Missing'))

print('Dims', X2.shape)
X2.fillna(-1, inplace=True)
model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, X2, y, cv=kf, scoring=rmsle).mean()
print('RMSLE:', error)


# End Measure time elapsed
end = timer()
print('\n')
print('Time Elapsed: {}'.format(timedelta(seconds = end - start)))

Dims (1460, 79)
RMSLE: 0.14383736485915208


Time Elapsed: 0:01:49.150778


### Bonus: OneHot Encoding

In [11]:
from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...


#from sklearn.preprocessing import OneHotEncoder
X3 = X.copy()
cats = []
for col in X3.columns:
    if X3[col].dtype == object:
        X3 = X3.join(pd.get_dummies(X3[col], prefix=col), how='left')
        X3.drop(col, axis=1, inplace=True)
    

print('Dims', X3.shape)
X3.fillna(-1, inplace=True)
model = RandomForestRegressor(n_estimators=1000, random_state=0)
cross_val_score(model, X3, y, cv=kf, scoring=rmsle).mean()

# End Measure time elapsed
end = timer()
print('\n')
print('Time Elapsed: {}'.format(timedelta(seconds = end - start)))

Dims (1460, 288)


Time Elapsed: 0:03:05.116613


## Target Transformations

An interesting way to create diversity, and sometimes even better performance, in a regression case is to transform the variable we are trying to predict. In this case we will test two transformations: logarithm and square root.

### Log

In [12]:
from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...


model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, X1, np.log1p(y), cv=kf, scoring=rmsle_log_y).mean()
print('RF, X1, log-target RMSLE:', error)

model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, X2, np.log1p(y), cv=kf, scoring=rmsle_log_y).mean()
print('RF, X2, log-target RMSLE:', error)



# End Measure time elapsed
end = timer()
print('\n')
print('Time Elapsed: {}'.format(timedelta(seconds = end - start)))

RF, X1, log-target RMSLE: 0.14516026930052636
RF, X2, log-target RMSLE: 0.14209480789088058


Time Elapsed: 0:03:05.672931


### Square root


In [13]:
from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...


model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, X1, np.sqrt(y), cv=kf, scoring=rmsle_sqrt_y).mean()
print('RF, X1, sqrt-target RMSLE:', error)

model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, X2, np.sqrt(y), cv=kf, scoring=rmsle_sqrt_y).mean()
print('RF, X2, sqrt-target RMSLE:', error)


# End Measure time elapsed
end = timer()
print('\n')
print('Time Elapsed: {}'.format(timedelta(seconds = end - start)))

RF, X1, sqrt-target RMSLE: 0.14565293448427205
RF, X2, sqrt-target RMSLE: 0.14300460013198157


Time Elapsed: 0:03:11.847593


## Generating models with different models / algorithms


In [14]:
from sklearn.ensemble import GradientBoostingRegressor

### GBM + Log

In [15]:
from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...


model = GradientBoostingRegressor(random_state=0)
error = cross_val_score(model, X1, np.log1p(y), cv=kf, scoring=rmsle_log_y).mean()
print('GBM, X1, log-target RMSLE:', error)

model = GradientBoostingRegressor(random_state=0)
error = cross_val_score(model, X2, np.log1p(y), cv=kf, scoring=rmsle_log_y).mean()
print('GBM, X2, log-target RMSLE:', error)


# End Measure time elapsed
end = timer()
print('\n')
print('Time Elapsed: {}'.format(timedelta(seconds = end - start)))

GBM, X1, log-target RMSLE: 0.13349245491356668
GBM, X2, log-target RMSLE: 0.12980689048155075


Time Elapsed: 0:00:06.705459


### GBM + Square root

In [16]:
from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...


model = GradientBoostingRegressor(random_state=0)
error = cross_val_score(model, X1, np.sqrt(y), cv=kf, scoring=rmsle_sqrt_y).mean()
print('GBM, X1, sqrt-target RMSLE:', error)

model = GradientBoostingRegressor(random_state=0)
error = cross_val_score(model, X2, np.sqrt(y), cv=kf, scoring=rmsle_sqrt_y).mean()
print('GBM, X2, sqrt-target RMSLE:', error)


# End Measure time elapsed
end = timer()
print('\n')
print('Time Elapsed: {}'.format(timedelta(seconds = end - start)))

GBM, X1, sqrt-target RMSLE: 0.13466955860674396
GBM, X2, sqrt-target RMSLE: 0.13107304131368688


Time Elapsed: 0:00:06.640773


### Adjusting hyperparameters

## Stacking

In [17]:
from itertools import product
from sklearn.linear_model import Ridge

In [18]:
from timeit import default_timer as timer
from datetime import timedelta

# Start Measure time elapsed
start = timer()
# Code here ...


kf_out = KFold(n_splits=5, shuffle=True, random_state=1)
kf_in = KFold(n_splits=5, shuffle=True, random_state=2)

cv_mean = []
for fold, (tr, ts) in tqdm(enumerate(kf_out.split(X, y))):
    X1_train, X1_test = X1.iloc[tr], X1.iloc[ts]
    X2_train, X2_test = X2.iloc[tr], X2.iloc[ts]
    y_train, y_test = y.iloc[tr], y.iloc[ts]
    
    modelos = [GradientBoostingRegressor(random_state=0), RandomForestRegressor(random_state=0)]
    targets = [np.log1p, np.sqrt]
    feature_sets = [(X1_train, X1_test), (X2_train, X2_test)]
    
    
    predictions_cv = []
    predictions_test = []
    for model, target, feature_set in tqdm(product(modelos, targets, feature_sets)):
        predictions_cv.append(cross_val_predict(model, feature_set[0], target(y_train), cv=kf_in).reshape(-1,1))
        model.fit(feature_set[0], target(y_train))
        ptest = model.predict(feature_set[1])
        predictions_test.append(ptest.reshape(-1,1))
    
    predictions_cv = np.concatenate(predictions_cv, axis=1)
    predictions_test = np.concatenate(predictions_test, axis=1)
    
    stacker = Ridge()
    stacker.fit(predictions_cv, np.log1p(y_train))
    error = rmsle_log_y(stacker, predictions_test, np.log1p(y_test))
    cv_mean.append(error)
    print('RMSLE Fold %d - RMSLE %.4f' % (fold, error))
    
print('RMSLE CV5 %.4f' % np.mean(cv_mean))
    

# End Measure time elapsed
end = timer()
print('\n')
print('Time Elapsed: {}'.format(timedelta(seconds = end - start)))

RMSLE Fold 0 - RMSLE 0.1251
RMSLE Fold 1 - RMSLE 0.1440
RMSLE Fold 2 - RMSLE 0.1267
RMSLE Fold 3 - RMSLE 0.1378
RMSLE Fold 4 - RMSLE 0.1078

RMSLE CV5 0.1283


Time Elapsed: 0:04:14.186327
